In [1]:
#Load packages
import numpy as np
import os
import scipy.io as sio
from scipy.special import gammaln, softmax  
from scipy.linalg import solve_triangular, solve, eigh, eig
from itertools import groupby
import pandas as pd
import random
import scipy
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from numpy.linalg import matrix_rank
from scipy.stats import multivariate_t
from sklearn.model_selection import train_test_split
from sklearn.cluster import AgglomerativeClustering

In [2]:
#Load data
whole = sio.loadmat('whole_wolabels.mat')
parts = sio.loadmat('parts_wolabels.mat')

In [3]:
#functions
def encode_labels(labels):
    le = LabelEncoder()
    le.fit(labels)
    encoded_labels = le.transform(labels)
    
    return encoded_labels, le

def decode_labels(encoded_predict_labels, le):
    test_predictions = le.inverse_transform(encoded_predict_labels)
    
    return test_predictions

def Harmonic_Mean(ytest, ypred, ytrain):
    ytest = np.array(ytest)
    ypred = np.array(ypred)
    true_dict = Counter(ytest)
    prediction_dict = Counter(ypred)
    uytest = set(ytest) #all classes present during testing may contain unseen classes 
                        # and only a subset of the seen classes
    uytrain = set(ytrain) #classes represented during training, seen classes
    
    #Mean class accuracy
    uy = list(uytest.intersection(uytrain))
    nc = len(uy)
    Classification_accuracy = np.zeros(nc)
    for i in range(nc):
        Index_of_i_th_class_true = list(np.argwhere(ytest == uy[i]).ravel())
        Index_of_i_th_class_pred = list(np.argwhere(ypred == uy[i]).ravel())
        tp = len(list(set(Index_of_i_th_class_true).intersection(Index_of_i_th_class_pred)))
        Classification_accuracy[i] = tp/true_dict[uy[i]]
    Classification_accuracy = np.mean(Classification_accuracy)
    
    #Out of distribution f1 score
    uy = set(uy)
    u_unseen_classes = list(uytest - uy)
    Index_of_unseen_class_true = list(np.argwhere(np.isin(ytest, u_unseen_classes)).ravel())
    Index_of_unseen_class_pred = list(np.argwhere(ypred > 5000).ravel())
    tp = len(list(set(Index_of_unseen_class_true).intersection(Index_of_unseen_class_pred)))
    fp = len(Index_of_unseen_class_pred) - tp
    fn = len(Index_of_unseen_class_true) - tp
    OOD_F1 = (2*tp)/(2*tp + fp + fn)
    
    harmonic_mean = 2*Classification_accuracy*OOD_F1/(Classification_accuracy+OOD_F1) 
                                    #harmonic mean of the two scores
    
    return Classification_accuracy, OOD_F1, harmonic_mean 

### Calculating class mean and covariance priors ###
def calculate_priors(xtrain, ytrain):
    d = xtrain.shape[1]
    uy = np.unique(ytrain)
    nc = len(uy)
    mu0 = np.zeros((1,d))
    Sigma0 = np.zeros((d,d))
    for i in range(nc):
        idx = (ytrain==uy[i]).flatten()
        if len(xtrain[idx,:]) == 1:
            mu0 += xtrain[idx,:] 
        else:
            mu0 = mu0 + np.mean(xtrain[idx,:], axis=0)
            Sigma0 = Sigma0 + np.cov(xtrain[idx,:], rowvar=False)
    mu0 = mu0/nc
    Sigma0 = Sigma0/nc
    
    return np.squeeze(mu0), Sigma0
    
### Calculating Posterior Predictive Distribution parameters ###
def calculate_ppd_params(xtrain, ytrain, unseenclasses, Psi, mu0, m, k0, k1):

    cnt = 0
    seenclasses = np.unique(ytrain)
    nc   = len(seenclasses) + len(unseenclasses)
    n, d = xtrain.shape

    Sig_s    = np.zeros((d,d,nc))
    Sigmas   = np.zeros((d,d,nc))
    mu_s     = np.zeros((nc, d))
    v_s      = np.zeros((nc, 1), dtype=np.int32)
    class_id = np.zeros((nc, 1))
    
    # The first part: for Seen classes
    uy              = seenclasses
    ncl             = len(uy)

    for i in range(ncl):
        idx         = np.in1d(ytrain, uy[i])
        Xi          = xtrain[idx]

        # The current selected component stats: # points, mean and scatter
        cur_n       = np.sum(idx)
        #print(cur_n)
        cur_S       = (cur_n-1)*np.cov(Xi.T)
        #print(cur_S)
        cur_mu      = np.mean(Xi, axis=0, keepdims=True)
        
        # The case where only data likelihood and global priors are used and local priors are ignored. This 
        # is the case we used for seen classes as mentioned in the paper
        v_s[cnt]        = cur_n+m-d+1
        mu_s[cnt]       = (cur_n*cur_mu+(k0*k1/(k0+k1))*mu0)/(cur_n+(k0*k1/(k0+k1)))
        Smu             = ((cur_n*(k0*k1/(k0+k1)))/((k0*k1/(k0+k1))+cur_n))*np.dot(cur_mu-mu0, (cur_mu-mu0).T)
        if cur_n == 1:
            Sig_s[:,:,cnt]  = (Psi+Smu)/(((cur_n+(k0*k1/(k0+k1)))*v_s[cnt])/(cur_n+(k0*k1/(k0+k1))+1))
        else:
            Sig_s[:,:,cnt]  = (Psi+cur_S+Smu)/(((cur_n+(k0*k1/(k0+k1)))*v_s[cnt])/(cur_n+(k0*k1/(k0+k1))+1))
        if np.isnan(np.sum(Sig_s[:,:,i])):
            print(i, "th cov is nan")
            
        class_id[cnt]   = uy[i]
        cnt            += 1

    # The second part: creating local priors for each Genus
    ncl           = len(unseenclasses)
    
    # Main for loop for  Genus local priors params estimation
    for i in range(ncl):
        classes = unseenclasses[i]
        #Extract corresponding data
        idx       = np.in1d(ytrain, classes)
        Yi        = ytrain[idx]
        Xi        = xtrain[idx]
        uyi       = np.unique(Yi)

        # Initialize component sufficient statistics 
        ncpi      = len(uyi)
        xkl       = np.zeros((ncpi,d))      # Component means
        Skl       = np.zeros((d,d,ncpi))    # Component scatter matrices
        kap       = np.zeros((ncpi,1))      # model specific
        nkl       = np.zeros((ncpi,1))      # number of data points in the components

        # Calculate  sufficient statistics for each component in meta cluster
        for j in range(ncpi):
            idx        = np.in1d(Yi, uyi[j])
            nkl[j]     = np.sum(idx)
            kap[j]     = nkl[j]*k1/(nkl[j]+k1)
            Xij        = Xi[idx]
            xkl[j]     = np.mean(Xij, axis=0)
            if nkl[j] > 1:
                Skl[:,:,j] = (nkl[j]-1)*np.cov(Xij.T)   

        # Model specific parameters
        sumkap       = np.sum(kap)
        kaps         = (sumkap+k0)*k1/(sumkap+k0+k1)
        sumSkl       = np.sum(Skl,axis=2)
        muk          = (np.sum(np.multiply(xkl, kap*np.ones((1,d))), axis=0)+k0*mu0)/(sumkap+k0)
        
        # Unseen classes' predictive cov, mean and dof
        v_s[cnt]         = np.sum(nkl)-ncpi+m-d+1
        class_id[cnt]   = 5000 + cnt
        Sigmas[:,:,cnt] = Psi+sumSkl
        Sig_s[:,:,cnt]  = (Psi+sumSkl)/((kaps*v_s[cnt])/(kaps+1))
        if np.isnan(np.sum(Sig_s[:,:,cnt])):
            print(cnt, "th cov is nan")
            print(Psi)
            print(sumSkl)
        mu_s[cnt]       = muk
        cnt             += 1  

    return Sig_s, mu_s, v_s, class_id, Sigmas 

### PPD calculation (Log-Likelihood of Student-t) ###
def bayesian_cls_evaluate(xtest, all_loc_vec, all_scale_mat, all_dof, all_labels):
    nr_classes = len(all_labels)
    log_lik = np.zeros((xtest.shape[0], nr_classes))
    for i in range(nr_classes):
        log_lik[:, i] = multivariate_t.logpdf(xtest, all_loc_vec[i], \
                                              all_scale_mat[:, :, i], all_dof[i])
    
    predicted_class_ids = np.argmax(log_lik, axis=1)
    ypred = np.zeros(len(xtest))
    
    for i in range(len(predicted_class_ids)):
        ypred[i] = all_labels[predicted_class_ids[i]]
    
    return ypred

def bayesian_cls_train(x_tr, y_tr, unseenclasses, k_0=0.1, k_1=10, m=5*500, mu_0=0, s=1):
    s_classes   = np.unique(y_tr)   
    d0          = x_tr.shape[1] 
    [mu_0, Sigma_0] = calculate_priors(x_tr, y_tr)
    #print("Sigma_0:", Sigma_0)
    Psi=(m-d0-1)*Sigma_0/s
    #print("Psi:", Psi)

    # Class predictive cov, mean and DoF from unconstrained model
    #print('PPD derivation is Done!!')
    return calculate_ppd_params(x_tr, y_tr, unseenclasses, Psi, mu_0, m, k_0, k_1) 

In [4]:
#whole

#train
train_classid = np.squeeze(whole['train_classid'])
train_class_labels = []
for item in train_classid:
    train_class_labels.append(item[0])
train_features = whole['train_feats']
train_imid = whole['train_imgid']
train_imgid = []
for item in train_imid:
    train_imgid.append(item[0])
train_imgid = np.squeeze(train_imgid)
train_sampleid = whole['train_sampleid']

#validation
validation_classid = np.squeeze(whole['val_classid'])
validation_class_labels = []
for item in validation_classid:
    validation_class_labels.append(item[0])
validation_features = whole['val_feats']
validation_imid = whole['val_imgid']
validation_imgid = []
for item in validation_imid:
    validation_imgid.append(item[0])
validation_imgid = np.squeeze(validation_imgid)
validation_sampleid = whole['val_sampleid']

#test
test_features = whole['test_feats']
test_imid = whole['test_imgid']
test_imgid = []
for item in test_imid:
    test_imgid.append(item[0])
test_imgid = np.squeeze(test_imgid)
test_sampleid = whole['test_sampleid']

#encoded train labels
train_labels, le = encode_labels(train_class_labels)
train_unique_labels = sorted(np.unique(train_labels))
train_unique_labels_count = len(train_unique_labels)
print(len(train_labels))

#encoded validation labels
validation_labels = le.transform(validation_class_labels)
validation_unique_labels = sorted(np.unique(validation_labels))
validation_unique_labels_count = len(validation_unique_labels)
print(len(validation_labels))

7849
1379


In [5]:
#parts

#train
train_classid_parts = np.squeeze(parts['train_classid'])
train_class_labels_parts = []
for item in train_classid_parts:
    train_class_labels_parts.append(item[0])
train_features_parts = parts['train_feats']
train_imid_parts = parts['train_imgid']
train_imgid_parts = []
for item in train_imid_parts:
    train_imgid_parts.append(item[0])
train_imgid_parts = np.squeeze(train_imgid_parts)
train_sampleid_parts = parts['train_sampleid']
train_tileid_parts = parts['train_tileid']

#validation
validation_classid_parts = np.squeeze(parts['val_classid'])
validation_class_labels_parts = []
for item in validation_classid_parts:
    validation_class_labels_parts.append(item[0])
validation_features_parts = parts['val_feats']
validation_imid_parts = parts['val_imgid']
validation_imgid_parts = []
for item in validation_imid_parts:
    validation_imgid_parts.append(item[0])
validation_imgid_parts = np.squeeze(validation_imgid_parts)
validation_sampleid_parts = parts['val_sampleid']
validation_tileid_parts = parts['val_tileid']

#test
test_features_parts = parts['test_feats']
test_imid_parts = parts['test_imgid']
test_imgid_parts = []
for item in test_imid_parts:
    test_imgid_parts.append(item[0])
test_imgid_parts = np.squeeze(test_imgid_parts)
test_sampleid_parts = parts['test_sampleid']
test_tileid_parts = parts['test_tileid']

Minmax normalization

In [6]:
scaler = MinMaxScaler()
train_features_norm = scaler.fit_transform(train_features)
validation_features_norm = scaler.transform(validation_features)

Creating holdout dataset

In [7]:
# combine train and validation data labels
total_labels = np.array(list(train_labels) + list(validation_labels))
print(len(total_labels))

validation_count_dict = Counter(validation_labels)
#print(validation_count_dict)

validation_labels_dict = sorted(validation_count_dict.items(), key=lambda x: x[1], reverse=True)
unique_unseen_labels = []
for i in range(75,125):
      unique_unseen_labels.append(validation_labels_dict[i][0])
print("unique_unseen_labels:", unique_unseen_labels)

unseen_labels_index = np.argwhere(np.isin(total_labels, unique_unseen_labels)).ravel()
print("unseen_labels length:", len(unseen_labels_index))

#separating unseen classes from train data
unseen_labels_index_train = np.argwhere(np.isin(train_labels, unique_unseen_labels)).ravel()
unseen_labels_train = train_labels[unseen_labels_index_train]
unseen_embeddings_train = train_features_norm[unseen_labels_index_train]

#separating seen classes from train data
unique_seen_labels = sorted(list(set(train_unique_labels) - set(unique_unseen_labels)))
print("unique_seen_labels length:", len(unique_seen_labels))

seen_labels_index_train = np.argwhere(np.isin(train_labels, unique_seen_labels)).ravel()
print("seen_labels index train length:", len(seen_labels_index_train))
seen_labels_train = train_labels[seen_labels_index_train].reshape(len(seen_labels_index_train)).tolist()
seen_embeddings_train = train_features_norm[seen_labels_index_train]

#Final train data
xtrain = seen_embeddings_train
ytrain = np.array(seen_labels_train)

#final validation data
xtest = np.vstack((validation_features_norm, unseen_embeddings_train))
ytest = np.concatenate((validation_labels, unseen_labels_train))

#print(train_embeddings)
print(xtrain.shape)
#print(train_labels)
print(len(ytrain))
#print(validation_embeddings)
print(xtest.shape)
#print(validation_labels)
print(len(ytest))

9228
unique_unseen_labels: [49, 51, 60, 61, 78, 84, 86, 88, 89, 100, 103, 106, 107, 110, 111, 113, 116, 121, 132, 134, 145, 146, 148, 152, 154, 158, 168, 175, 177, 199, 205, 207, 209, 212, 221, 222, 225, 227, 232, 236, 241, 249, 251, 255, 256, 257, 272, 273, 275, 281]
unseen_labels length: 706
unique_seen_labels length: 963
seen_labels index train length: 7243
(7243, 384)
7243
(1985, 384)
1985


cluster classes by genus

In [8]:
train_class_labels = decode_labels(ytrain, le)
print(len(train_class_labels))

train_unique_class_labels = sorted(np.unique(train_class_labels))

# sort list
# essential for grouping
train_unique_class_labels.sort()

# using lambda + itertools.groupby() + split() to group similar substrings
genus_cluster = [list(i) for j, i in groupby(train_unique_class_labels, lambda a: a.split(' ')[0])]

genus_cluster_encoded = []
for i in range(len(genus_cluster)):
    genus_cluster_encoded.append(list(le.transform(genus_cluster[i])))

print(len(genus_cluster_encoded))

7243
389


Train and evaluation: hyper parameter tuning

In [9]:
cluster_list = genus_cluster_encoded
d = xtrain.shape[1]

all_kappa_0s = [0.1, 1, 10]
all_kappa_1s = [0.01, 0.1, 1, 10]
all_ms = [d+2, 100*d]
all_s = [1, 10]

for k_0 in all_kappa_0s:
    for k_1 in all_kappa_1s:
        for m in all_ms:
            for s in all_s:
                ### PPD parameter estimation ###
                Sig_s, mu_s, v_s, class_id, _= bayesian_cls_train(xtrain, ytrain, cluster_list, k_0=k_0, k_1=k_1, m=m, s=s)

                ### Prediction phase ###
                ypred = bayesian_cls_evaluate(xtest, mu_s, Sig_s, v_s, class_id) 

                Classification_accuracy, OOD_F1, harmonic_mean = Harmonic_Mean(ytest, ypred, ytrain)

                print('Results from k0=%.2f, k1=%.2f, m=%d, s=%.1f:' % (k_0, k_1, m, s))
                print('Mean class acc: %.2f OOD F1: %.2f, Harmonic mean: %.2f'%(Classification_accuracy, OOD_F1, harmonic_mean))

/home/ntalukde/anaconda3/envs/NephNet3D/lib/python3.7/site-packages/ipykernel_launcher.py:93: RuntimeWarning: Degrees of freedom <= 0 for slice
/home/ntalukde/anaconda3/envs/NephNet3D/lib/python3.7/site-packages/numpy/lib/function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
/home/ntalukde/anaconda3/envs/NephNet3D/lib/python3.7/site-packages/numpy/lib/function_base.py:2542: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


Results from k0=0.10, k1=0.01, m=386, s=1.0:
Mean class acc: 0.81 OOD F1: 0.27, Harmonic mean: 0.41
Results from k0=0.10, k1=0.01, m=386, s=10.0:
Mean class acc: 0.69 OOD F1: 0.48, Harmonic mean: 0.57
Results from k0=0.10, k1=0.01, m=38400, s=1.0:
Mean class acc: 0.79 OOD F1: 0.04, Harmonic mean: 0.07
Results from k0=0.10, k1=0.01, m=38400, s=10.0:
Mean class acc: 0.01 OOD F1: 0.52, Harmonic mean: 0.03
Results from k0=0.10, k1=0.10, m=386, s=1.0:
Mean class acc: 0.63 OOD F1: 0.51, Harmonic mean: 0.57
Results from k0=0.10, k1=0.10, m=386, s=10.0:
Mean class acc: 0.58 OOD F1: 0.49, Harmonic mean: 0.53
Results from k0=0.10, k1=0.10, m=38400, s=1.0:
Mean class acc: 0.79 OOD F1: 0.13, Harmonic mean: 0.22
Results from k0=0.10, k1=0.10, m=38400, s=10.0:
Mean class acc: 0.00 OOD F1: 0.53, Harmonic mean: 0.00
Results from k0=0.10, k1=1.00, m=386, s=1.0:
Mean class acc: 0.55 OOD F1: 0.40, Harmonic mean: 0.46
Results from k0=0.10, k1=1.00, m=386, s=10.0:
Mean class acc: 0.54 OOD F1: 0.37, Harmoni

Observation:
    
Results from k0=0.01, k1=0.01, m=770, s=1.0:
Mean class acc: 0.73 OOD F1: 0.53, Harmonic mean: 0.61

Results from k0=0.01, k1=0.01, m=770, s=10.0:
Mean class acc: 0.57 OOD F1: 0.57, Harmonic mean: 0.57

Results from k0=0.01, k1=1.00, m=7680, s=1.0:
Mean class acc: 0.63 OOD F1: 0.60, Harmonic mean: 0.62

Results from k0=0.01, k1=1.00, m=768000, s=1.0:
Mean class acc: 0.64 OOD F1: 0.59, Harmonic mean: 0.61

Results from k0=0.01, k1=1.00, m=76800000, s=1.0:
Mean class acc: 0.64 OOD F1: 0.58, Harmonic mean: 0.61

Results from k0=0.01, k1=10.00, m=7680, s=1.0:
Mean class acc: 0.54 OOD F1: 0.59, Harmonic mean: 0.56

Results from k0=0.10, k1=0.01, m=770, s=1.0:
Mean class acc: 0.81 OOD F1: 0.48, Harmonic mean: 0.60

Results from k0=0.10, k1=0.01, m=770, s=10.0:
Mean class acc: 0.74 OOD F1: 0.57, Harmonic mean: 0.65

Results from k0=0.10, k1=0.01, m=770, s=100.0:
Mean class acc: 0.70 OOD F1: 0.58, Harmonic mean: 0.64

Results from k0=0.10, k1=0.10, m=770, s=1.0:
Mean class acc: 0.63 OOD F1: 0.57, Harmonic mean: 0.60

Results from k0=0.10, k1=1.00, m=7680, s=1.0:
Mean class acc: 0.63 OOD F1: 0.60, Harmonic mean: 0.62



1. For m > 1e3*d harmonic mean drops to 0.
2. For s> 10 harmonic mean drops significantly
3. k1 >=10 then m= 1e5*d is okay, but s>1 drives harmonic mean down
4. High k1 is bad for harmonic mean
5. s=10 seems good for outlier detection